In [1]:
%%capture

!pip install tensorflow_hub
!pip install tensorflow --upgrade
!mamba install -qy tqdm

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
print(tf. __version__)
import skillsnetwork
from tensorflow import keras
from tensorflow.keras import layers
# from keras.preprocessing.text import Tokenizer
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding,Masking,LSTM, GRU, Conv1D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SimpleRNN
from tensorflow.keras.datasets import reuters
from sklearn.model_selection import train_test_split
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
# import tensorflow_hub as hub


# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

sns.set_context('notebook')
sns.set_style('white')
np.random.seed(2024)

2.16.1


In [2]:
# from keras.preprocessing.text import Tokenizer
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow_hub as hub

In [3]:
def calculate_results(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred)
    model_precision, model_recall, model_f1,_ = precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall" :model_recall,
                     "f1":model_f1}
    return model_results

## RNN Fundamentals

RNNs fall in the category of neural networks that maintain some kind of **state**. They can process sequential data of arbitrary length. By doing so, they overcome certain limitations faced by classical neural networks. Classical NNs only accept fixed-length vectors as input and output fixed-length vectors. RNNs operate over sequences of vectors. Classical NNs aren't built to consider the sequential nature of some data. RNNs work with sequential data forms like language, video frames, time series, and so on.

The RNN layer uses a for-loop to iterate over the time-steps of a sequence, and maintains an internal state that encodes information about all time-steps that have been observed so far. The Keras RNN API has built-in `keras.layers.RNN` and `keras.layers.LSTM` layers that make it easy to quickly build RNN models.


In [4]:
train_df = pd.read_csv('./dataset/train.csv')

In [5]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [6]:
X_train, X_test, y_train, y_test = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(), test_size=0.1, random_state=42)

In [7]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None, 
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=None
)

In [8]:
max_vocab_length= 10000
max_length = 15

In [9]:
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                            )

In [10]:
lstm_layer = layers.LSTM(units=max_length)

In [11]:
encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                               input_shape=[],
                               dtype = tf.string,
                               trainable=False,
                               name="pretrained")

The `encoder_layer` will take as input variable length English text and the output is a 512 dimensional vector.


We will add a Dense layer with unit 1 to create a simple binary text classifier on top of any TF-Hub module. Next, we will compile and fit it using 20 epochs.


In [13]:
encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=True)

# Define your Sequential model
model = tf.keras.Sequential([
    layers.Input(shape=[], dtype=tf.string),  # Input layer for string inputs
    encoder_layer,  # TensorFlow Hub layer wrapped in Lambda layer
    layers.Dense(1, activation="sigmoid")  # Output layer
], name="model_pretrained")


: 

In [ ]:
model.compile(loss="binary_crossentropy",
                     optimizer="adam",
                     metrics=["accuracy"])

In [ ]:
model.fit(x=X_train,
              y=y_train,
              epochs=20,
              validation_data=(X_test,y_test))

In [ ]:
calculate_results(y_true=y_test,
                  y_pred=tf.squeeze(tf.round(model.predict(X_test))))